In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch 
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl

np.random.seed(123)

In [2]:
ratings = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv',
                     parse_dates=['timestamp'])

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [4]:
ratings.shape

(20000263, 4)

In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userId     int64         
 1   movieId    int64         
 2   rating     float64       
 3   timestamp  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 610.4 MB


In [6]:
rand_userIds = np.random.choice(ratings['userId'].unique(),
                               size=int(len(ratings['userId'].unique())*0.3),
                               replace=False)

ratings = ratings.loc[ratings['userId'].isin(rand_userIds)]

print('There are {} rows of data from {} users'.format(len(ratings),len(rand_userIds)))

There are 6027314 rows of data from 41547 users


In [7]:
ratings.shape

(6027314, 4)

In [8]:
ratings.sample(10)

,userId,movieId,rating,timestamp
3840312,26182,3704,4.0,2007-01-31 21:56:52
7608731,52439,3365,4.0,2004-03-21 08:02:56
19363634,134060,1027,3.0,2003-07-15 22:43:45
17181947,118860,2629,1.0,2007-11-29 21:27:08
9344779,64638,4723,2.0,2001-09-10 20:11:41
10356404,71637,3882,0.5,2004-02-20 01:48:53
962965,6460,11,4.0,1996-10-22 12:53:32
12014375,82949,54001,4.5,2010-08-06 16:39:25
10532053,72855,7158,2.5,2011-07-25 01:07:58
18710288,129551,1228,4.5,2007-06-28 00:17:42


In [9]:
ratings['rank_latest'] = ratings.groupby(['userId'])['timestamp'] \
                                .rank(method = 'first',ascending=False)

In [10]:
ratings.head()

,userId,movieId,rating,timestamp,rank_latest
236,3,1,4.0,1999-12-11 13:36:47,81.0
237,3,24,3.0,1999-12-14 12:54:08,10.0
238,3,32,4.0,1999-12-11 13:14:07,140.0
239,3,50,5.0,1999-12-11 13:13:38,143.0
240,3,160,3.0,1999-12-14 12:54:08,11.0


In [11]:
train_ratings = ratings[ratings['rank_latest'] != 1]
test_ratings = ratings[ratings['rank_latest'] == 1]

# drop columns that we no Longer need 
train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId','movieId','rating']]

In [12]:
ratings.head()

,userId,movieId,rating,timestamp,rank_latest
236,3,1,4.0,1999-12-11 13:36:47,81.0
237,3,24,3.0,1999-12-14 12:54:08,10.0
238,3,32,4.0,1999-12-11 13:14:07,140.0
239,3,50,5.0,1999-12-11 13:13:38,143.0
240,3,160,3.0,1999-12-14 12:54:08,11.0


In [13]:
train_ratings.loc[:, 'rating'] = 1

train_ratings.sample(5)

,userId,movieId,rating
3411906,23263,5481,1.0
1815983,12245,5464,1.0
16198592,112109,6,1.0
13914487,96124,1247,1.0
1445807,9790,1690,1.0


In [14]:
# # Get a list of all movie IDs
# all_movieIds = ratings['movieId'].unique()

# # Placeholfers that will hold the training data
# users, items, labels = [], [], []

# # This is the set of items that each usesr has interaction with
# user_item_set = set(zip(train_ratings['userId'],train_ratings['movieId']))

# # 4:1 ratio of negative to positive samples
# num_negatives = 4

# for (u, i) in tqdm(user_item_set):
#     users.append(u)
#     items.append(i)
#     labels.append(1) # items that the user has interacted with are positive
#     for _ in range(num_negatives):
#         # randomly select an item
#         negative_item = np.random.choice(all_movieIds)
#         # check that the user has not interacted with this item
#         while (u, negative_item) in user_item_set:
#             negative_item = np.random.choice(all_movieIds)
#         users.append(u)
#         items.append(negative_item)
#         labels.append(0) # items not interacted with are negative

In [15]:
class MovieLens20MDataset(Dataset):
    def __init__(self, user_ids, movie_ids, num_negatives= 4):
        self.users, self.items, self.labels = self.get_dataset(user_ids, movie_ids, num_negatives)
        
    def __len__(self):
        return len(self.users)
    
    def __getitem__ (self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]
    
    def get_dataset(self, user_ids, movie_ids, num_negatives):
        # This is the set of items that each usesr has interaction with
        user_item_set = set(zip(user_ids, movie_ids))
        users, items, labels = [], [], []
        all_movieIds = movie_ids.unique()
        for (u, i) in tqdm(user_item_set):
            users.append(u)
            items.append(i)
            labels.append(1) # items that the user has interacted with are positive
            for _ in range(num_negatives):
                # randomly select an item
                negative_item = np.random.choice(all_movieIds)
                # check that the user has not interacted with this item
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_movieIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0) # items not interacted with are negative
        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)
        

In [16]:
train_dataset = MovieLens20MDataset(train_ratings['userId'], train_ratings['movieId'])
for i, sample in enumerate(train_dataset):
    print(i, sample[0].size(), sample[1].size())

    if i == 3:
        break

  0%|          | 0/5985767 [00:00<?, ?it/s]

0 torch.Size([]) torch.Size([])
1 torch.Size([]) torch.Size([])
2 torch.Size([]) torch.Size([])
3 torch.Size([]) torch.Size([])
